In [14]:
import sys
from os.path import abspath, exists
from os import makedirs
import pandas as pd
from pandas_datareader import data as pdr
import yfinance as yf
yf.pdr_override()

ROOT_DIR = abspath('../')
sys.path.append(ROOT_DIR)
ROOT_DIR

'/home/mhmunem/finapps/BB_Backtest'

In [36]:
SP500_URL = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
TICKER_FILE_PATH = f"{ROOT_DIR}/data/sp500_tickers.csv"
START_DATE = "2011-01-01"
END_DATE = "2023-12-31"
CSVS_FILE_PATH = f"{ROOT_DIR}/data/sp500/daily/"
PARQUET_FILE_PATH = f"{ROOT_DIR}/data/sp500_stock_data.parquet"

In [37]:
def clean_data(dataframe: pd.DataFrame) -> pd.DataFrame:
    df = dataframe.reset_index()
    # Format the data as per the specified format
    df = df[["Date", "Open", "High", "Low", "Close", "Volume"]]
    df = df.rename(columns={'Date':'timestamp', 'Open':'open', 'High':'high', 
                            'Low':'low','Close':'close', 'Volume':'volume'}) # type: ignore
    
    df["timestamp"] = df["timestamp"].dt.strftime('%Y-%m-%d %H:%M:%S.%f')
    # Round numeric columns to 2 decimal places
    numeric_cols = ["open", "high", "low", "close", "volume"]
    df[numeric_cols] = df[numeric_cols].round(2) 
    df['open'] = df['open'].astype(float)
    df['high'] = df['high'].astype(float)
    df['low'] = df['low'].astype(float)
    df['close'] = df['close'].astype(float)
    df['volume'] = df['volume'].astype(float)
    df = df.dropna()
    df = df.reindex(columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'], index=df.index[::-1])
    df.reset_index(drop=True, inplace=True)
    df.set_index(keys='timestamp', inplace=True)
    return df


def download_tickers():
    data = pd.read_html(SP500_URL)
    sp500_tickers = data[0]['Symbol'].str.replace('.', '-').replace('RVTY (Previously PKI)', 'RVTY').tolist()
    sp500_df = pd.DataFrame(sp500_tickers, columns=['Ticker'])
    print(f"Downloading...")
    # Save to CSV
    sp500_df.to_csv(TICKER_FILE_PATH, index=False)
    print(f"Downloaded")
    

def download_csvs():
    sp500_df = pd.read_csv(TICKER_FILE_PATH)
    sp500_tickers = sp500_df['Ticker'].tolist()
    
    if not exists(CSVS_FILE_PATH):
        makedirs(CSVS_FILE_PATH)

    for ticker in sp500_tickers:
        try:
            data = pdr.get_data_yahoo(ticker, start=START_DATE, end=END_DATE, progress=False)
            data = clean_data(data)
            data.to_csv(f"{CSVS_FILE_PATH}{ticker}.csv")
            print(f"Downloaded data for {ticker}")
        except Exception as e:
            print(f"Error downloading data for {ticker}: {e}")


def download_parquets():
    # Read the CSV file
    sp500_df = pd.read_csv(TICKER_FILE_PATH)
    sp500_tickers = sp500_df['Ticker'].tolist()

    # download dataframe
    sp500_stock_data = pdr.get_data_yahoo(sp500_tickers, start=START_DATE, end=END_DATE)
    sp500_stock_data = sp500_stock_data.rename(columns={'Date':'timestamp', 'Open':'open', 'High':'high', 
                                'Low':'low','Close':'close', 'Volume':'volume'}) # type: ignore
    sp500_stock_data = sp500_stock_data.drop(columns="Adj Close")
    # Swap levels 0 and 1
    sp500_stock_data = sp500_stock_data.swaplevel(0, 1, axis=1)
    #Sort the columns by the levels
    sp500_stock_data = sp500_stock_data.sort_index(axis=1, level=[0, 1])
    #save to parquet file
    sp500_stock_data.to_parquet(PARQUET_FILE_PATH, engine="pyarrow")
    
def download_data(type: str):
    if type == "sp500tickers":
        download_tickers()
    elif type == "sp500csvs":
        download_csvs()
    elif type == "sp500parquet":
        download_parquets()
    

In [39]:
if __name__ == "__main__":
    # download_data("sp500parquet")
    print("running...")

[*********************100%%**********************]  503 of 503 completed
